In [1]:
from bs4 import BeautifulSoup
import urllib 
import re
from urllib.parse import unquote
import string
import dateutil.parser as dateparser
from time import sleep
from pprint import pprint
import os
import json
import datetime


folder = os.path.join('C:/', 'users', 'charr', 'documents', 'soundtrack')
letters = list(string.ascii_uppercase)

In [2]:
def get_image(soup):
    try:
        return [[[td.get('src') for td in tr.find_all('img')] for tr in table.find_all('tr')][0] for table in soup.find_all('table')[2:]][0][0]
    except:
        return'No Image Available'
    
        
def get_album_info(soup):
    try:
        alb = [[[td for td in tr.find_all('td')] for tr in table.find_all('tr')] for table in soup.find_all('table')[3:]][1]
        album_info = {}
        for l in alb:
            if len(l) > 1:
                for i in l:
                    if i.get('class') == ['infoline']:
                        info = i.get_text(strip=True).split(":")
                        album_info[info[0]] = info[1:]
            if len(l) == 1:
                if l[0].get('class') == ['composerline']:
                    album_info['composer'] = l[0].get_text()
        return album_info
    except:
        return'No Album Information'

def get_song_info(soup):
    try:
        songs = [[[td for td in tr.find_all('td')] for tr in table.find_all('tr')] for table in soup.find_all('table')][2:][0]

        index = []
        for j,l in enumerate(songs):
            for i,x in enumerate(l):
                if x.get_text() == 'Album Tracklist':
                    index.append(i)

        song_list = []
        for i in range(index[0]-1, len(songs)-5):
            song_list.append(songs[i])

        final_song_list = []
        for i in song_list:
            if len(i) > 1:
                final_song_list.append(i[1].get_text(strip=True).split("-"))

        return final_song_list
    except:
        return 'No Song Data Available'
        
def get_soundtrack_information(soup):
    info_dct = {'img': get_image(soup), 
                'album': get_album_info(soup), 
                'songs': get_song_info(soup)}
    return info_dct

In [3]:
url_list = []
for letter in letters:
    html_text = urllib.request.urlopen(f"http://www.soundtrackinfo.com/lettersearch/{letter}/").read()
    soup = BeautifulSoup(html_text, 'html.parser')

    for i in soup.find_all(['td', 'tr', 'a'])[80:]:
        if i.get('href') != None:
            url_list.append(i.get('href'))
            
#url_list

In [4]:
len(url_list)

12458

In [5]:
def save_tweet_dict(dct):
    file_suffix = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = 'soundtrack_data' + file_suffix + '.json'
    filepath = os.path.join(folder, filename)
    with open(filepath, mode='w', encoding = 'utf-8') as f:
        json.dump(dct, f)

In [8]:
soundtrack_dct = {}
error_list = []
for url in url_list:
    if len(soundtrack_dct) > 1000:
        save_tweet_dict(soundtrack_dct)
        soundtrack_dct = {}
    else: 
        try:
            if "/lettersearch/only" not in url:
                html_text = urllib.request.urlopen(f"http://www.soundtrackinfo.com/tracks{url[4:]}").read()
                soup = BeautifulSoup(html_text, 'html.parser')

                soundtrack_dct[url[4:].strip('/')] = get_soundtrack_information(soup)
        except:
            error_list.append(url)


In [9]:
#final soundtrack_dct for last group of <1000 movies
save_tweet_dict(soundtrack_dct)

In [10]:
len(error_list)

0